# Eventos y señales

Recordemos que las interfaces gráficas son aplicaciones construidas con una **arquitectura basada en manejo de eventos**. Esto significa que la aplicación detecta eventos generados por el usuario o por otras partes de la misma aplicación, y los procesa con el elemento apropiado. 

Una aplicación construida con PyQt empieza a detectar eventos una vez que ha entrado en su *mainloop*, y esto ocurre luego de llamar al método ```exec_()```. La siguiente figura muestra una comparación, mediante diagramas de flujo, entre un programa con una estructura lineal como los que habíamos construido en las semanas anteriores, y un programa con uso de GUI basada en manejo de eventos.

![](img/GUI-flowchart.png)

En este modelo existen 3 elementos fundamentales:

- **La fuente del evento**: Corresponde al objeto que genera el cambio de estado o que genera el evento
- **El objeto evento**: Es el objeto que encapsula el cambio de estado mediante el evento. 
- **El objeto destino:** Equivale al objeto que se desea notificar del cambio de estado

Bajo este modelamiento, la fuente del evento **delega** la tarea de manejar el evento al objeto de destino. PyQt utiliza un mecanismo de **signal** y **slot** para manejar los eventos y permitir comunicación entre _widgets_. Cuando un evento ocurre, el objeto que es activado emite una señal (_signal_) al _slot_ correspondiente, donde un _slot_ puede ser cualquier tipo de elemento llamable (_callable_) en Python.

En el siguiente ejemplo, extenderemos el programa anterior que mostraba botones dispuestos en grilla que simulaban una calculadora, para generar una llamada a la función `boton_clickeado` cada vez que se presiona alguno de los botones. Esto se logra conectando el evento que enviará la señal con el _slot_ que la recibe. En el caso de los botones, generalemente el método corresponde al evento `clicked`. Mediante el método `connect()` se establece la comunicación entre el evento y el _slot_. Este método recibe una función _llamable_ en Python, (recordemos no agregar los `()`). El  ejemplo solo muestra la clase `MiVentana`.

In [ ]:
class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_GUI()

    def init_GUI(self):
        self.label1 = QLabel('', self)
        self.grilla = QGridLayout()

        valores = ['1', '2', '3',
                   '4', '5', '6',
                   '7', '8', '9',
                   '0', 'CE', 'C']

        posicion = [(i, j) for i in range(4) for j in range(3)]

        for posicion, valor in zip(posicion, valores):
            if valor == '':
                continue
            boton = QPushButton(valor)

            """
            Aquí conectamos el evento clicked() de cada boton con el slot
            correspondiente. En este ejemplo todos los botones usan el
            mismo slot.
            """
            boton.clicked.connect(self.boton_clickeado)

            self.grilla.addWidget(boton, *posicion)

        vbox = QVBoxLayout()
        vbox.addWidget(self.label1)
        vbox.addLayout(self.grilla)
        self.setLayout(vbox)

        self.move(300, 150)
        self.setWindowTitle('Calculator')

    def boton_clickeado(self):
        """
        Esta función se ejecuta cada vez que uno de los botones de la grilla
        es presionado. Cada vez que el botón genera el evento, la función inspecciona
        cual botón fue presionado y recupera la posición en que se encuentra en
        la grilla.
        """

        # Sender retorna el objeto que fue clickeado.
        # Ahora, la variable boton referencia una instancia de QPushButton
        boton = self.sender()

        # Obtenemos el identificador del elemento en la grilla
        idx = self.grilla.indexOf(boton)

        # Con el identificador obtenemos la posición del ítem en la grilla
        posicion = self.grilla.getItemPosition(idx)

        # Actualizamos label1
        self.label1.setText(f'Presionado boton {idx}, en fila/columna: {posicion[:2]}.')


## Obtener al emisor de la señal: `sender`

En ocasiones, como en la función `boton_clickeado()` del código anterior, es necesario conocer cuál de los objetos del formulario envió una señal. El método `sender()` retorna el objeto que generó el evento.

El siguiente ejemplo también utiliza `sender()` para obtener una referencia al objeto que generó el evento. La clase `QtCoreApplication` permite obtener una referencia al objeto `QApplication` que está en ejecución, y forzar una salida. 

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel, QHBoxLayout, QVBoxLayout)
from PyQt5.QtCore import QCoreApplication


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_GUI()

    def init_GUI(self):
        self.label1 = QLabel('Status:', self)

        """
        El evento de cada botón es conectado con su slot. En este caso es 
        el mismo método boton_callback().
        """
        self.boton1 = QPushButton('&Boton 1', self)
        self.boton1.resize(self.boton1.sizeHint())
        self.boton1.clicked.connect(self.boton_callback)

        self.boton2 = QPushButton('&Boton 2', self)
        self.boton2.clicked.connect(self.boton_callback)
        self.boton2.resize(self.boton2.sizeHint())

        self.boton3 = QPushButton('&Salir', self)
        self.boton3.clicked.connect(QCoreApplication.instance().quit)
        self.boton3.resize(self.boton3.sizeHint())

        hbox = QHBoxLayout()
        hbox.addStretch(1)
        hbox.addWidget(self.boton1)
        hbox.addWidget(self.boton2)
        hbox.addWidget(self.boton3)
        hbox.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addWidget(self.label1)
        vbox.addLayout(hbox)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Agregamos todos los elementos al formulario
        self.setGeometry(200, 100, 300, 200)
        self.setWindowTitle('Sender')

    def boton_callback(self):
        # Esta función registra el objeto que envía la señal del evento
        # y lo refleja mediante el método sender() en label3.
        sender = self.sender()
        self.label1.setText(f'Status: presionado boton {sender.text()}')
        self.label1.resize(self.label1.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec_())


## Generar señales personalizadas

PyQt permite definir señales personalizadas por el usuario. Para esto se debe crear el objeto que alojará la nueva señal. Las señales son una subclase de `QtCore.QObject`. Dentro del objeto se crea la nueva señal como una instancia de ```QtCore.pyqtSignal```. Finalmente, en el _widget_ principal se deben crear la señal y las funciones que manejan la señal. 

El siguiente ejemplo muestra cómo generar una nueva señal que se activa al presionar alguna parte del _widget_ principal. Para emitir la señal se utiliza el método ```emit()``` heredado desde ```pyqtSignal()```.

In [ ]:
import sys

from PyQt5.QtCore import (QObject, pyqtSignal)
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class MiSignal(QObject):
    """
    Esta clase contiene las señales que permiten la comunicación entre
    elementos de la GUI.
    """
    escribe_signal = pyqtSignal()


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.inicializa_GUI()

    def inicializa_GUI(self):
        # Creamos un objeto para manejar las señales y conectamos el método
        # encargado de ejecutar la tarea
        self.s = MiSignal()
        self.s.escribe_signal.connect(self.escribe_etiqueta)

        self.etiqueta1 = QLabel('Etiqueta', self)
        self.etiqueta1.move(20, 10)
        self.resize(self.etiqueta1.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emit signal')
        self.show()

    def mousePressEvent(self, event):
        """
        Este evento maneja cuando se presiona alguno de los botones del
        mouse. Está asociado al evento correspondiente por defecto,
        y definido como una función que no hace nada. 
        Aquí hacemos override de él.
        """
        self.s.escribe_signal.emit()

    def escribe_etiqueta(self):
        self.etiqueta1.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta1.resize(self.etiqueta1.sizeHint())


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MiVentana()
    sys.exit(app.exec_())


### Señales personalizadas y _threads_

Otro uso de las señales personalizadas es en una aplicación _multithreaded_. El siguiente ejemplo muestra cómo crear una señal que controla la acción del _thread_ sobre el formulario mediante el método ``update_labels()``. El _thread_ por su parte, recibe como parámetro la señal y emite mensajes.

In [ ]:
import sys
from threading import Thread
from time import sleep

from PyQt5.QtCore import pyqtSignal
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QHBoxLayout,
                             QVBoxLayout, QPushButton)


class Evento:
    """
    Esta clase maneja el evento que será transmitido por la señal a su
    respectivo slot cada vez que se produzca la emisión. Por simplicidad este
    evento solo incluye un mensaje, pero en la medida que se requiera podría
    portar más información.
    """

    def __init__(self, msg=''):
        self.msg = msg


class MiThread(Thread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, trigger_signal, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.trigger = trigger_signal

    def run(self):
        # Creamos un evento para transmitir datos desde el thread a la interfaz
        evento = Evento()

        for i in range(10):
            sleep(0.5)
            evento.msg = str(i)
            self.trigger.emit(evento)

        evento.msg = 'Status: thread terminado'
        self.trigger.emit(evento)


class MiVentana(QWidget):

    # Creamos una señal para manejar la respuesta del thread
    threads_response = pyqtSignal(object)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()
        self.thread = None

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel('Status: esperando thread', self)
        self.boton = QPushButton('Start Thread', self)
        self.boton.clicked.connect(self.start_threads)

        hbox1 = QHBoxLayout()
        hbox1.addStretch(1)
        hbox1.addWidget(self.label)
        hbox1.addStretch(1)

        hbox2 = QHBoxLayout()
        hbox2.addStretch(1)
        hbox2.addWidget(self.boton)
        hbox2.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addLayout(hbox1)
        vbox.addStretch(1)
        vbox.addLayout(hbox2)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Conectamos la señal del thread al método que maneja
        self.threads_response.connect(self.update_labels)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle('Ejemplo threads')
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def start_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        if self.thread is None or not self.thread.is_alive():
            self.thread = MiThread(self.threads_response)
            self.thread.start()

    def update_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento.msg)


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())


## Eventos de _mouse_ y teclado

Las maneras más comunes en que una interfaz gráfica genera eventos por parte del usuario,
son a través del teclado y el _mouse_.
Cuando un usuario **hace clic** o **presiona una tecla** ocurren eventos para los que muchas veces queremos definir un comportamiento. La clase `QWidget` incluye los métodos `mousePressEvent()` y `keyPressEvent()`, que se hacen cargo del comportamiento del programa ante cada uno de estos eventos, respectivamente. Por defecto, estos métodos no hacen nada, pero siempre podemos hacer _override_ de ellos en nuestros propios _widgets_.


In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.inicializa_GUI()

    def inicializa_GUI(self):
        self.etiqueta1 = QLabel('Etiqueta', self)
        self.etiqueta1.move(20, 10)
        self.resize(self.etiqueta1.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emit signal')
        self.show()

    def mousePressEvent(self, event):
        """
        Este evento maneja cuando se presiona alguno de los botones del mouse.
        """
        self.etiqueta1.setText('Presionaron el mouse')
        self.etiqueta1.resize(self.etiqueta1.sizeHint())

    def keyPressEvent(self, event):
        """
        Este método maneja el evento que se produce al presionar las teclas.
        """
        self.etiqueta1.setText(f'Presionaron la tecla: {event.text()}')
        self.etiqueta1.resize(self.etiqueta1.sizeHint())


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MiVentana()
    sys.exit(app.exec_())
